In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

# Funções auxiliares
def Adiciona_Link_em_Lista (link_atual, lista_links, categoria):
    """
    Função void que vai adicionar 'link_atual' em 'lista_links' na devida 'categoria'.
    Se o 'link_atual' já estiver dentro da 'lista_links', então verifica se a 'categoria' já está na 'lista_links'

    Param: 
        link_atual: str com o link do app
        lista_links: lista que contém os links dos apps e as categorias a que pertencem
            (ex. [[link1, [categoria1, categoria2]], [link33, [categoria3]]])
        categoria: str com a categoria que está sendo buscada

    Return:
        void
    """
    ja_registrado = False
    for registro in lista_links:
        if (link_atual in registro):
            if (categoria not in registro[1]):
                registro[1].append(categoria)
            ja_registrado = True
    
    if (not ja_registrado):
        lista_links.append([link_atual, [categoria]])

def Extrai_Apps_em_Alta (element, lista_final):
    colunas_apps = element.find_elements(By.XPATH, './/c-wiz/section/div/div[3]/div/div/div/div[1]/div')
    #print(f"Quantidade de colunas: {len(colunas_apps)}")

    for coluna in colunas_apps:
        #print(f"Coluna número: {cont}")
        apps = coluna.find_elements(By.XPATH, './div')
        #print(f"Quantidade de apps encontrados: {len(apps)}")
        for app in apps:
            try:
                classificacao = app.find_element(By.XPATH, './/div/a/div[1]/div').text
                #print(classificacao)
                link = app.find_element(By.XPATH, './/div/a').get_attribute("href")
                #print(link)
                categoria = app.find_element(By.XPATH, './/div/a/div[2]/div/div[2]/span').text
                #print(categoria)
                lista_final.append([link, categoria, classificacao])
            except:
                print("Deu ruim")

def Formata_Dimensao (leitura):
    multiplicador = 1
    numero = float(leitura[0].replace(',', '.'))
    if (leitura[1] == 'mil'):
        multiplicador *= 10**3
    elif (leitura[1] == 'mi'):
        multiplicador *= 10**6
    elif (leitura[1] == 'bi'):
        multiplicador *= 10**9
        
    return int(numero * multiplicador)

def Formata_Data(data):
    data = data.replace(".", "").split()
    dic_meses = {
        "jan":"01",
        "fev":"02",
        "mar":"03",
        "abr":"04",
        "mai":"05",
        "jun":"06",
        "jul":"07",
        "ago":"08",
        "set":"09",
        "out":"10",
        "nov":"11",
        "dez":"12"
    }
    dia = data[0]
    mes = dic_meses[data[2]]
    ano = data[-1]
    data_final = dia+'/'+mes+"/"+ano
    return data_final

In [2]:
# BLOCO QUE VAI ABRIR O CHROME E ENTRAR NO SITE DA PLAY STORE

driver = webdriver.Chrome() #Criando uma instância do Google Chrome e salvando na variável 'driver'
link_google_play = 'https://play.google.com/store/apps'
driver.get(link_google_play) #Navegando para o URL de interesse

In [3]:
# BLOCO QUE VAI CRIAR AS LISTAS COM O LINK DOS APPS EM ALTA

#Localizando todas as divisões que contêm as categorias de Apps
xpath_element = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/c-wiz/div/c-wiz/c-wiz'
elements = driver.find_elements(By.XPATH, xpath_element)
print("Elementos localizados.")

#Procurando a divisão 'Em Alta'
lista_apps_gratuitos = []
lista_apps_rentaveis = []
lista_apps_pagos = []
for element in elements:
    try:
        titulo = element.find_element(By.XPATH, ".//c-wiz/section/header/div").text
        #print(categoria)
        if(titulo == "Em alta"):
            print("Extraindo os Apps gratuitos em alta")
            Extrai_Apps_em_Alta (element, lista_apps_gratuitos)

            botao_rentaveis = '//*[@id="ct|apps_topgrossing"]'
            driver.find_element(By.XPATH, botao_rentaveis).click()
            sleep(2)
            print("Extraindo os Apps mais rentáveis.")
            Extrai_Apps_em_Alta (element, lista_apps_rentaveis)

            botao_pagos = '//*[@id="ct|apps_topselling_paid"]'
            driver.find_element(By.XPATH, botao_pagos).click()
            sleep(2)
            print("Extraindo os Apps pagos em alta")
            Extrai_Apps_em_Alta (element, lista_apps_pagos)
    
    except:
        # print("Elemento sem categoria.")
        # print(element.text)
        pass

# Exportar essas três listas para um bloco de notas (será usado em outro programa)
lista_geral = lista_apps_gratuitos + lista_apps_pagos + lista_apps_rentaveis
arquivo = open("lista_apps.txt", "w")
for app in lista_geral:
    linha = app[0] + "," + app[1] + "," + app[2] + "\n"
    arquivo.write(linha)
arquivo.close()



Elementos localizados.
Extraindo os Apps gratuitos em alta
Extraindo os Apps mais rentáveis.
Extraindo os Apps pagos em alta


In [4]:
# Após obter os links dos Apps em alta, entrar em cada link e extrair algumas informações
def Extrai_Dados (driver, lista_apps):
    lista_geral = []
    for app in lista_apps:
        #print(f"Lendo o app de classificação: {app[2]}")
        link = app[0]
        categoria = app[1]
        classificacao = app[2]
        
        driver.get(link)
        sleep(2)

        xpath_nome = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/h1/span'
        nome_app = driver.find_element(By.XPATH, xpath_nome).text

        xpath_cabecalho = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[2]/div/div/div'
        infos_introdutorias = driver.find_elements(By.XPATH, xpath_cabecalho)
        nota_app = 0
        num_avaliacoes = 0
        for info in infos_introdutorias:
            leitura = info.find_element(By.XPATH, ".//div[2]").text
            for valor in leitura.split():
                if (valor == "avaliações"):
                    num_avaliacoes = Formata_Dimensao(leitura.split())
                    nota_app = info.find_element(By.XPATH, ".//div[1]/div/div").text.split()[0]
                    nota_app = float(nota_app.replace(",", "."))
        

        xpath_botao_instalar = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/div/div[1]/div/c-wiz/div/div/div/div/button/span'
        conteudo_botao = driver.find_element(By.XPATH, xpath_botao_instalar).text
        tipo = 'Gratuito'
        preco = 0
        if (conteudo_botao != "Instalar"):
            tipo = "Pago"
            preco = float(conteudo_botao.split()[2][:-1].replace(",", "."))

        contem_anuncios = "Não"
        compras_no_app = "Não"
        try:
            driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]')
            content1 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[1]').text
            if (content1 == "Contém anúncios"):
                contem_anuncios = "Sim"
            elif (content1 == "Compras no app"):
                compras_no_app = "Sim"
            
            try:
                content2 = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[1]/div/div/c-wiz/div[2]/div[1]/div/div/div[2]/div/span[2]').text
                if (content1 == "Contém anúncios"):
                    contem_anuncios = "Sim"
                elif (content1 == "Compras no app"):
                    compras_no_app = "Sim"
            except:
                pass
        except:
            pass

        height = 300
        driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo um pouco a página para clicar em "Sobre este app"

        mudou_de_pagina = False
        while (not mudou_de_pagina):
            try:
                xpath_botao_infos = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[2]/div/section/header/div/div[2]/button'
                driver.find_element(By.XPATH, xpath_botao_infos).click() #Abrindo "Sobre este app"
                mudou_de_pagina = True
            except:
                height += 300
                driver.execute_script("window.scrollTo(0," + str(height) + ")") # Descendo mais um pouco a página
        sleep(1)

        informacoes = driver.find_elements(By.XPATH, '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div[3]/div')
        
        #Inicializando variáveis
        versao_app = ''
        ultima_atualizacao = ''
        android_min = ''
        num_downloads_min = ''
        classificacao_usuarios = ''
        data_lancamento = ''

        for info in informacoes:
            titulo_campo = info.find_element(By.XPATH, "./div[1]").text
            valor_campo = info.find_element(By.XPATH, "./div[2]").text
            match titulo_campo:
                case "Versão":
                    versao_app = valor_campo
                case "Atualizado em":
                    ultima_atualizacao = valor_campo
                    ultima_atualizacao = Formata_Data(ultima_atualizacao)
                case "Requer Android":
                    android_min = valor_campo.split()[0]
                case "Downloads":
                    num_downloads_min = valor_campo.split()[0].replace("+", "")
                    num_downloads_min = int(num_downloads_min.replace(".", ""))
                case "Classificação do conteúdo":
                    classificacao_usuarios = valor_campo.replace("Saiba mais", "").strip()
                case "Lançado em":
                    data_lancamento = valor_campo
                    data_lancamento = Formata_Data(data_lancamento)

        lista_geral.append([nome_app, categoria, classificacao, num_downloads_min, tipo, preco, nota_app, num_avaliacoes, contem_anuncios, compras_no_app, versao_app, ultima_atualizacao, android_min, classificacao_usuarios, data_lancamento])
    
    colunas = ["App", "Categoria", "Ranking", "Downloads", "Tipo", "Preco", "Nota", "Avaliacoes", "Contem_Anuncios", "Compras_no_App", "Versao", "Ultima_Atualizacao", "Android", "Classificacao_Conteudo", "Lancamento"]
    df = pd.DataFrame(lista_geral, columns=colunas)

    return df
        
print("Extraindo os dados relativos aos Apps Gratuitos...")
df_gratuitos = Extrai_Dados (driver, lista_apps_gratuitos)

print("Extraindo os dados relativos aos Apps mais rentáveis...")
df_rentaveis = Extrai_Dados (driver, lista_apps_rentaveis)

print("Extraindo os dados relativos aos Apps Pagos...")
df_pagos = Extrai_Dados (driver, lista_apps_pagos)

# Exportando os DataFrames para um arquivo .csv
df_gratuitos.to_csv("top45_apps_gratuitos.csv", index=False)
df_rentaveis.to_csv("top45_apps_rentaveis.csv", index=False)
df_pagos.to_csv("top45_apps_pagos.csv", index=False)

driver.close()

Extraindo os dados relativos aos Apps Gratuitos...
Lendo o app de classificação: 1
Lendo o app de classificação: 2
Lendo o app de classificação: 3
Lendo o app de classificação: 4
Lendo o app de classificação: 5
Lendo o app de classificação: 6
Lendo o app de classificação: 7
Lendo o app de classificação: 8
Lendo o app de classificação: 9
Lendo o app de classificação: 10
Lendo o app de classificação: 11
Lendo o app de classificação: 12
Lendo o app de classificação: 13
Lendo o app de classificação: 14
Lendo o app de classificação: 15
Lendo o app de classificação: 16
Lendo o app de classificação: 17
Lendo o app de classificação: 18
Lendo o app de classificação: 19
Lendo o app de classificação: 20
Lendo o app de classificação: 21
Lendo o app de classificação: 22
Lendo o app de classificação: 23
Lendo o app de classificação: 24
Lendo o app de classificação: 25
Lendo o app de classificação: 26
Lendo o app de classificação: 27
Lendo o app de classificação: 28
Lendo o app de classificação: 29
L

In [6]:
df_gratuitos

,App,Categoria,Ranking,Downloads,Tipo,Preco,Nota,Avaliacoes,Contem_Anuncios,Compras_no_App,Versao,Ultima_Atualizacao,Android,Classificacao_Conteudo,Lancamento
0,Shopee: Compre de Tudo Online,Compras,1,100000000,Gratuito,0,4.5,3360000,Não,Não,2.91.32,13/08/2022,4.4,Classificação Livre,20/10/2019
1,SHEIN-Compras de Moda Online,Compras,2,100000000,Gratuito,0,4.8,4120000,Não,Não,8.3.8,4/08/2022,5.0,"Classificação 14 anos • Conteúdo Sexual, Lingu...",18/03/2014
2,Scret: perguntas anônimas,Social,3,1000000,Gratuito,0,3.7,6040,Sim,Não,2.4,9/08/2022,5.0,Classificação Livre,15/07/2022
3,"Nubank: conta, cartão e mais",Finanças,4,50000000,Gratuito,0,4.7,1920000,Não,Não,Varia de acordo com o dispositivo,12/08/2022,Varia,Classificação Livre,25/04/2014
4,MegaFlix - Filmes e Séries,Entretenimento,5,1000000,Gratuito,0,4.6,54300,Sim,Não,2,21/07/2022,4.4,"Classificação 12 anos • Violência, Linguagem I...",21/07/2022
5,Auxílio Brasil GovBR,Social,6,10000000,Gratuito,0,3.5,146000,Não,Não,3.23.0,2/08/2022,5.0,Classificação Livre,23/09/2015
6,CAIXA Tem,Finanças,7,100000000,Gratuito,0,4.1,4710000,Não,Não,Varia de acordo com o dispositivo,14/08/2022,Varia,Classificação Livre,6/04/2020
7,phone cleaner,Ferramentas,8,10000000,Gratuito,0,3.8,7520,Sim,Não,1.5.1,9/07/2022,7.0,Classificação Livre,29/04/2022
8,Bravo Cleaner: Impulsionador,Ferramentas,9,10000000,Gratuito,0,4.2,41900,Sim,Não,1.3.4.1001,12/08/2022,5.0,Classificação Livre,21/09/2021
9,Kwai - Ver Vídeos Bacanas,Reproduzir e editar vídeos,10,100000000,Gratuito,0,4.8,9430000,Sim,Não,6.7.10.526806,15/08/2022,5.0,Classificação 12 anos • Linguagem Imprópria,2/03/2018


In [7]:
df_rentaveis

,App,Categoria,Ranking,Downloads,Tipo,Preco,Nota,Avaliacoes,Contem_Anuncios,Compras_no_App,Versao,Ultima_Atualizacao,Android,Classificacao_Conteudo,Lancamento
0,Google One,Produtividade,1,500000000,Gratuito,0,4.3,367000,Não,Sim,Varia de acordo com o dispositivo,25/07/2022,Varia,Classificação Livre,12/09/2018
1,"Globoplay: filmes, séries e +",Entretenimento,2,50000000,Gratuito,0,4.3,667000,Sim,Não,Varia de acordo com o dispositivo,10/08/2022,5.0,"Classificação 14 anos • Violência, Temas Sensí...",20/08/2013
2,HBO Max: Assista à TV e filmes,Entretenimento,3,100000000,Gratuito,0,3.1,397000,Sim,Não,Varia de acordo com o dispositivo,8/08/2022,5.0,"Classificação 14 anos • Violência, Conteúdo Se...",29/06/2021
3,Tinder - Bate Papo e Namoro,Estilo de vida,4,100000000,Gratuito,0,3.5,4700000,Sim,Não,13.14.0,2/08/2022,7.0,"Classificação 14 anos • Conteúdo Sexual, Lingu...",15/07/2013
4,Disney+,Entretenimento,5,100000000,Gratuito,0,4.3,1970000,Não,Sim,2.9.1-rc1,15/08/2022,5.0,"Classificação 16 anos • Violência, Conteúdo Se...",16/11/2020
5,"Canva: Design, Foto e Vídeo",Arte e design,6,100000000,Gratuito,0,4.8,9150000,Não,Sim,2.178.0,10/08/2022,5.0,Classificação 10 anos • Linguagem Imprópria,27/11/2017
6,TikTok,Social,7,1000000000,Gratuito,0,4.5,51200000,Sim,Não,Varia de acordo com o dispositivo,15/08/2022,Varia,Classificação 12 anos • Linguagem Imprópria,14/09/2015
7,Star+,Entretenimento,8,10000000,Gratuito,0,4.3,213000,Não,Sim,2.10.0-rc1,1/08/2022,5.0,"Classificação 16 anos • Violência, Conteúdo Se...",17/08/2021
8,Crunchyroll,Entretenimento,9,50000000,Gratuito,0,4.4,1020000,Sim,Não,Varia de acordo com o dispositivo,10/08/2022,Varia,"Classificação 16 anos • Violência, Conteúdo Se...",20/12/2010
9,Kwai - Ver Vídeos Bacanas,Reproduzir e editar vídeos,10,100000000,Gratuito,0,4.8,9430000,Sim,Não,6.7.10.526806,15/08/2022,5.0,Classificação 12 anos • Linguagem Imprópria,2/03/2018


In [8]:
df_pagos

,App,Categoria,Ranking,Downloads,Tipo,Preco,Nota,Avaliacoes,Contem_Anuncios,Compras_no_App,Versao,Ultima_Atualizacao,Android,Classificacao_Conteudo,Lancamento
0,Novo Hinário Adventista,Música e áudio,1,1000,Pago,10.99,4.1,130,Não,Não,1.0.3,8/08/2022,6.0,Classificação Livre,8/07/2022
1,Game Booster 4x Faster Pro,Ferramentas,2,1000000,Pago,1.29,4.4,97700,Não,Sim,1.1.8,5/08/2022,4.1,Classificação Livre,3/11/2019
2,Panda Mouse Pro(BETA),Ferramentas,3,100000,Pago,11.99,3.5,9250,Não,Não,1.5.0,2/04/2021,4.4,Classificação Livre,23/10/2019
3,Panda Gamepad Pro (BETA),Ferramentas,4,500000,Pago,12.00,2.9,30900,Não,Não,1.4.9,2/04/2021,4.4,Classificação Livre,21/02/2019
4,Torque Pro (OBD2 / Carro),Comunicação,5,1000000,Pago,11.95,3.9,71600,Não,Não,1.12.98,7/03/2022,4.1,Classificação Livre,7/03/2010
5,Você tem 1 real?,Entretenimento,6,10000,Pago,0.99,4.7,504,Não,Não,2.3,22/05/2021,4.1,Classificação Livre,19/01/2020
6,Meta Launcher PRO - iOS 15,Personalização,7,50000,Pago,0.49,4.5,7480,Sim,Não,31,30/03/2022,5.0,Classificação Livre,10/02/2022
7,Nova Launcher Prime,Personalização,8,5000000,Pago,4.99,4.6,357000,Não,Não,Varia de acordo com o dispositivo,22/12/2020,Varia,Classificação Livre,23/02/2012
8,Toca Life: Hospital,Educação,9,500000,Pago,12.90,4.2,7200,Sim,Não,1.3-play,27/06/2022,4.4,Classificação Livre,13/04/2017
9,audioPro™ B̶R̶L̶8̶.̶9̶9̶,Música e áudio,10,100000,Pago,0.69,4.3,4800,Não,Não,9.4.8,25/01/2021,4.4,Classificação Livre,24/12/2019
